## Gather the 2D fields for CESM2

In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import glob
import scipy
import copy
import math
import time

#plotting with Cartopy. 
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from matplotlib import cm
from matplotlib import rc
#rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
rc('font',**{'family':'serif','serif':['Times']})
rc('text', usetex=True)
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from matplotlib import ticker


# import scipy
from datetime import datetime
import os
# import utils
import importlib


from scipy import stats
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

from shapely.geometry.polygon import LinearRing
from dask.diagnostics import ProgressBar
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import matplotlib.patches as mpatches
import math
import statsmodels.api as sm
from statsmodels.graphics import tsaplots
import geocat.comp as gcomp
import shutil
import metpy.calc as mpcalc


In [2]:
if 'client' in locals():
    client.shutdown()
    print('...shutdown client...')
else:
    print('client does not exist yet')

client does not exist yet


In [17]:
from distributed import Client
from ncar_jobqueue import NCARCluster

cluster = NCARCluster(project='NAML0001',walltime='12:00:00')
cluster.scale(40)
client = Client(cluster)
client

## FLUT

In [4]:
FNSflut = sorted(glob.glob('/glade/campaign/collections/cmip/CMIP6/timeseries-cmip6/b.e21.B1850.f09_g17.CMIP6-esm-piControl.001/atm/proc/tseries/day_1/*FLUT.01*'))
DSFLUT = xr.open_mfdataset(FNSflut)
DSFLUT.to_netcdf("/glade/scratch/wchapman/data_for_KJM/FLUT_train_dask.nc")

FNSflut = sorted(glob.glob('/glade/campaign/collections/cmip/CMIP6/timeseries-cmip6/b.e21.B1850.f09_g17.CMIP6-esm-piControl.001/atm/proc/tseries/day_1/*FLUT.02*'))
DSFLUT = xr.open_mfdataset(FNSflut)
DSFLUT.to_netcdf("/glade/scratch/wchapman/data_for_KJM/FLUT_val_dask.nc")


FNSflut = sorted(glob.glob('/glade/campaign/collections/cmip/CMIP6/timeseries-cmip6/b.e21.B1850.f09_g17.CMIP6-esm-piControl.001/atm/proc/tseries/day_1/*FLUT.03*'))
DSFLUT = xr.open_mfdataset(FNSflut)
DSFLUT.to_netcdf("/glade/scratch/wchapman/data_for_KJM/FLUT_test_dask.nc")

## Z500

In [5]:
FNSflut = sorted(glob.glob('/glade/campaign/collections/cmip/CMIP6/timeseries-cmip6/b.e21.B1850.f09_g17.CMIP6-esm-piControl.001/atm/proc/tseries/day_1/*Z500.01*'))
DSFLUT = xr.open_mfdataset(FNSflut)
DSFLUT.to_netcdf("/glade/scratch/wchapman/data_for_KJM/Z500_train_dask.nc")

FNSflut = sorted(glob.glob('/glade/campaign/collections/cmip/CMIP6/timeseries-cmip6/b.e21.B1850.f09_g17.CMIP6-esm-piControl.001/atm/proc/tseries/day_1/*Z500.02*'))
DSFLUT = xr.open_mfdataset(FNSflut)
DSFLUT.to_netcdf("/glade/scratch/wchapman/data_for_KJM/Z500_val_dask.nc")

FNSflut = sorted(glob.glob('/glade/campaign/collections/cmip/CMIP6/timeseries-cmip6/b.e21.B1850.f09_g17.CMIP6-esm-piControl.001/atm/proc/tseries/day_1/*Z500.03*'))
DSFLUT = xr.open_mfdataset(FNSflut)
DSFLUT.to_netcdf("/glade/scratch/wchapman/data_for_KJM/Z500_test_dask.nc")

## U Winds

In [4]:
FNSflut = sorted(glob.glob('/glade/campaign/collections/cmip/CMIP6/timeseries-cmip6/b.e21.B1850.f09_g17.CMIP6-esm-piControl.001/atm/proc/tseries/day_1/*.U.01*'))
DSFLUT = xr.open_mfdataset(FNSflut).sel(lev=[200,850],method='nearest')
DSFLUT.to_netcdf("/glade/scratch/wchapman/data_for_KJM/U_train_dask.nc")

FNSflut = sorted(glob.glob('/glade/campaign/collections/cmip/CMIP6/timeseries-cmip6/b.e21.B1850.f09_g17.CMIP6-esm-piControl.001/atm/proc/tseries/day_1/*.U.02*'))
DSFLUT = xr.open_mfdataset(FNSflut).sel(lev=[200,850],method='nearest')
DSFLUT.to_netcdf("/glade/scratch/wchapman/data_for_KJM/U_val_dask.nc")


FNSflut = sorted(glob.glob('/glade/campaign/collections/cmip/CMIP6/timeseries-cmip6/b.e21.B1850.f09_g17.CMIP6-esm-piControl.001/atm/proc/tseries/day_1/*.U.03*'))
DSFLUT = xr.open_mfdataset(FNSflut).sel(lev=[200,850],method='nearest')
DSFLUT.to_netcdf("/glade/scratch/wchapman/data_for_KJM/U_test_dask.nc")


In [7]:
FNSflut = sorted(glob.glob('/glade/campaign/collections/cmip/CMIP6/timeseries-cmip6/b.e21.B1850.f09_g17.CMIP6-esm-piControl.001/atm/proc/tseries/day_1/*.U.01*'))
chunk_sizes = {'time': 100, 'lat': 192, 'lon': 288}


for fnfn in FNSflut:
    DSFLUT = xr.open_dataset(fnfn, chunks=chunk_sizes).sel(lev=[200,850],method='nearest')
    DSFLUT.to_netcdf("/glade/scratch/wchapman/data_for_KJM/"+fnfn.split('/day_1/')[-1])    


FNSflut = sorted(glob.glob('/glade/campaign/collections/cmip/CMIP6/timeseries-cmip6/b.e21.B1850.f09_g17.CMIP6-esm-piControl.001/atm/proc/tseries/day_1/*.U.02*'))
chunk_sizes = {'time': 100, 'lat': 192, 'lon': 288}


for fnfn in FNSflut:
    DSFLUT = xr.open_dataset(fnfn, chunks=chunk_sizes).sel(lev=[200,850],method='nearest')
    DSFLUT.to_netcdf("/glade/scratch/wchapman/data_for_KJM/"+fnfn.split('/day_1/')[-1])    

    
FNSflut = sorted(glob.glob('/glade/campaign/collections/cmip/CMIP6/timeseries-cmip6/b.e21.B1850.f09_g17.CMIP6-esm-piControl.001/atm/proc/tseries/day_1/*.U.03*'))
chunk_sizes = {'time': 100, 'lat': 192, 'lon': 288}


for fnfn in FNSflut:
    DSFLUT = xr.open_dataset(fnfn, chunks=chunk_sizes).sel(lev=[200,850],method='nearest')
    DSFLUT.to_netcdf("/glade/scratch/wchapman/data_for_KJM/"+fnfn.split('/day_1/')[-1])    


In [8]:
FNSflut = sorted(glob.glob('/glade/scratch/wchapman/data_for_KJM/b.e21.B1850.f09_g17.CMIP6-esm-piControl.001*.U.01*'))
DSFLUT = xr.open_mfdataset(FNSflut).sel(lev=[200,850],method='nearest')
DSFLUT.to_netcdf("/glade/scratch/wchapman/data_for_KJM/U_train_dask.nc")

FNSflut = sorted(glob.glob('/glade/scratch/wchapman/data_for_KJM/b.e21.B1850.f09_g17.CMIP6-esm-piControl.001*.U.02*'))
DSFLUT = xr.open_mfdataset(FNSflut).sel(lev=[200,850],method='nearest')
DSFLUT.to_netcdf("/glade/scratch/wchapman/data_for_KJM/U_val_dask.nc")


FNSflut = sorted(glob.glob('/glade/scratch/wchapman/data_for_KJM/b.e21.B1850.f09_g17.CMIP6-esm-piControl.001*.U.03*'))
DSFLUT = xr.open_mfdataset(FNSflut).sel(lev=[200,850],method='nearest')
DSFLUT.to_netcdf("/glade/scratch/wchapman/data_for_KJM/U_test_dask.nc")


In [18]:
FNSflut = sorted(glob.glob('/glade/scratch/wchapman/data_for_KJM/b.e21.B1850.f09_g17.CMIP6-esm-piControl.001*.U.01*'))
DSFLUT = xr.open_mfdataset(FNSflut).sel(lev=[200,850],method='nearest')
DSFLUT.to_netcdf("/glade/scratch/wchapman/data_for_KJM/U_train_dask.nc")

FNSflut = sorted(glob.glob('/glade/scratch/wchapman/data_for_KJM/b.e21.B1850.f09_g17.CMIP6-esm-piControl.001*.U.02*'))
DSFLUT = xr.open_mfdataset(FNSflut).sel(lev=[200,850],method='nearest')
DSFLUT.to_netcdf("/glade/scratch/wchapman/data_for_KJM/U_val_dask.nc")


FNSflut = sorted(glob.glob('/glade/scratch/wchapman/data_for_KJM/b.e21.B1850.f09_g17.CMIP6-esm-piControl.001*.U.03*'))
DSFLUT = xr.open_mfdataset(FNSflut).sel(lev=[200,850],method='nearest')
DSFLUT.to_netcdf("/glade/scratch/wchapman/data_for_KJM/U_test_dask.nc")

FLUT_test_dask.nc   U_test_dask.nc   Z500_test_dask.nc
FLUT_train_dask.nc  U_train_dask.nc  Z500_train_dask.nc
FLUT_val_dask.nc    U_val_dask.nc    Z500_val_dask.nc


## SST

In [11]:
path = '/glade/campaign/cesm/collections/cesmLE/CESM-CAM5-BGC-LE/ocn/proc/tseries/daily/SST/'
svpath = '/glade/work/wchapman/For_Tony/'
sst_nc_1=path+'b.e11.B1850C5CN.f09_g16.005.pop.h.nday1.SST.05000101-05991231.nc'
xr.open_dataset(sst_nc_1)

<xarray.Dataset>
Dimensions:             (nlat: 384, nlon: 320, time: 36500, z_t: 60, z_w: 60,
                         d2: 2, z_t_150m: 15, z_w_bot: 60, z_w_top: 60)
Coordinates:
    TLAT                (nlat, nlon) float64 ...
    TLONG               (nlat, nlon) float64 ...
    ULAT                (nlat, nlon) float64 ...
    ULONG               (nlat, nlon) float64 ...
  * time                (time) object 0500-01-02 00:00:00 ... 0600-01-01 00:0...
  * z_t                 (z_t) float32 500.0 1.5e+03 ... 5.125e+05 5.375e+05
  * z_t_150m            (z_t_150m) float32 500.0 1.5e+03 ... 1.35e+04 1.45e+04
  * z_w                 (z_w) float32 0.0 1e+03 2e+03 ... 5e+05 5.25e+05
  * z_w_bot             (z_w_bot) float32 1e+03 2e+03 3e+03 ... 5.25e+05 5.5e+05
  * z_w_top             (z_w_top) float32 0.0 1e+03 2e+03 ... 5e+05 5.25e+05
Dimensions without coordinates: nlat, nlon, d2
Data variables: (12/51)
    ANGLE               (nlat, nlon) float64 ...
    ANGLET              (nlat, nlon) float64 ...
    DXT                 (nlat, nlon) float64 ...
    DXU                 (nlat, nlon) float64 ...
    DYT                 (nlat, nlon) float64 ...
    DYU                 (nlat, nlon) float64 ...
    ...                  ...
    sea_ice_salinity    float64 ...
    sflux_factor        float64 ...
    sound               float64 ...
    stefan_boltzmann    float64 ...
    time_bound          (time, d2) object ...
    vonkar              float64 ...
Attributes: (12/13)
    title:                     b.e11.B1850C5CN.f09_g16.005
    history:                   Sun Oct 13 19:21:24 2013: /glade/apps/opt/nco/...
    Conventions:               CF-1.0; http://www.cgd.ucar.edu/cms/eaton/netc...
    contents:                  Diagnostic and Prognostic Variables
    source:                    CCSM POP2, the CCSM Ocean Component
    revision:                  $Id: tavg.F90 41939 2012-11-14 16:37:23Z mlevy...
    ...                        ...
    start_time:                This dataset was created on 2013-05-28 at 02:3...
    cell_methods:              cell_methods = time: mean ==> the variable val...
    nsteps_total:              1952750
    tavg_sum:                  86400.0
    NCO:                       4.3.4
    nco_openmp_thread_number:  1

In [4]:
DSFLUT = xr.open_mfdataset(['/glade/campaign/collections/cmip/CMIP6/timeseries-cmip6/b.e21.B1850.f09_g17.CMIP6-esm-piControl.001/ocn/proc/tseries/day_1/b.e21.B1850.f09_g17.CMIP6-esm-piControl.001.pop.h.nday1.SST.01010102-01510101.nc',
'/glade/campaign/collections/cmip/CMIP6/timeseries-cmip6/b.e21.B1850.f09_g17.CMIP6-esm-piControl.001/ocn/proc/tseries/day_1/b.e21.B1850.f09_g17.CMIP6-esm-piControl.001.pop.h.nday1.SST.01510102-02010101.nc',
'/glade/campaign/collections/cmip/CMIP6/timeseries-cmip6/b.e21.B1850.f09_g17.CMIP6-esm-piControl.001/ocn/proc/tseries/day_1/b.e21.B1850.f09_g17.CMIP6-esm-piControl.001.pop.h.nday1.SST.02010102-02510101.nc',
'/glade/campaign/collections/cmip/CMIP6/timeseries-cmip6/b.e21.B1850.f09_g17.CMIP6-esm-piControl.001/ocn/proc/tseries/day_1/b.e21.B1850.f09_g17.CMIP6-esm-piControl.001.pop.h.nday1.SST.02510102-03010101.nc',
'/glade/campaign/collections/cmip/CMIP6/timeseries-cmip6/b.e21.B1850.f09_g17.CMIP6-esm-piControl.001/ocn/proc/tseries/day_1/b.e21.B1850.f09_g17.CMIP6-esm-piControl.001.pop.h.nday1.SST.03010102-03510101.nc',
'/glade/campaign/collections/cmip/CMIP6/timeseries-cmip6/b.e21.B1850.f09_g17.CMIP6-esm-piControl.001/ocn/proc/tseries/day_1/b.e21.B1850.f09_g17.CMIP6-esm-piControl.001.pop.h.nday1.SST.03510102-04010101.nc'],parallel=True)

print('merging')
DSsst = DSFLUT['SST'].to_dataset().chunk({'nlat':384,'nlon':320,'time':100})
DSrm = DSFLUT['REGION_MASK'].isel(time=10).to_dataset()
DSua = DSFLUT['UAREA'].isel(time=10).to_dataset()
DSta = DSFLUT['TAREA'].isel(time=10).to_dataset()
DSsst = xr.merge([DSsst,DSrm,DSua,DSta])
print('savings')
DSsst.to_netcdf("/glade/scratch/wchapman/data_for_KJM/SST_dask.nc")


In [ ]:
# #!!!!!uncomment this next line to do the whole dataset (there is another line to comment out below):!!!!
DSsst = xr.open_mfdataset(['/glade/work/wchapman/For_Tony/b.e21.B1850.f09_g17.CMIP6-esm-piControl.001.pop.h.nday1.SST.01010102-01510101.nc',
                         '/glade/work/wchapman/For_Tony//b.e21.B1850.f09_g17.CMIP6-esm-piControl.001.pop.h.nday1.SST.01510102-02010101.nc',
                         '/glade/work/wchapman/For_Tony//b.e21.B1850.f09_g17.CMIP6-esm-piControl.001.pop.h.nday1.SST.02010102-02510101.nc',
                         '/glade/work/wchapman/For_Tony/b.e21.B1850.f09_g17.CMIP6-esm-piControl.001.pop.h.nday1.SST.02510102-03010101.nc',
                         '/glade/work/wchapman/For_Tony//b.e21.B1850.f09_g17.CMIP6-esm-piControl.001.pop.h.nday1.SST.03010102-03510101.nc',
                         '/glade/work/wchapman/For_Tony//b.e21.B1850.f09_g17.CMIP6-esm-piControl.001.pop.h.nday1.SST.03510102-04010101.nc',
                          ],combine='by_coords',parallel=True)

# #!!!!!!uncomment the previous line to do the whole dataset (there is another line to comment out below):!!!

#AND! comment this next line to do the whole dataset: 
# DSsst = xr.open_dataset('/glade/work/wchapman/For_Tony/b.e11.B1850C5CN.f09_g16.005.pop.h.nday1.SST.05000101-05991231_V2.nc')


#AND! comment the previous line to do the whole dataset: 
print('...convert calendar...')
DSsst=DSsst.convert_calendar("standard", use_cftime=True)
print('...loading...')
# Specify the desired chunk sizes for each dimension
chunk_sizes = {'lat': 192, 'lon': 288, 'time': 100}  # -1 means keep the existing chunk size
# Rechunk the dataset
DSsst = DSsst.chunk(chunk_sizes)
# DSsst=DSsst.load()
DSsst=DSsst.load()

In [ ]:
DSsst.to_netcdf('/glade/work/wchapman/For_Tony//SST_camgrid_gathered.nc')

In [7]:
sst_list = ['/glade/work/wchapman/For_Tony/b.e21.B1850.f09_g17.CMIP6-esm-piControl.001.pop.h.nday1.SST.01010102-01510101.nc',
                         '/glade/work/wchapman/For_Tony//b.e21.B1850.f09_g17.CMIP6-esm-piControl.001.pop.h.nday1.SST.01510102-02010101.nc',
                         '/glade/work/wchapman/For_Tony//b.e21.B1850.f09_g17.CMIP6-esm-piControl.001.pop.h.nday1.SST.02010102-02510101.nc',
                         '/glade/work/wchapman/For_Tony/b.e21.B1850.f09_g17.CMIP6-esm-piControl.001.pop.h.nday1.SST.02510102-03010101.nc',
                         '/glade/work/wchapman/For_Tony//b.e21.B1850.f09_g17.CMIP6-esm-piControl.001.pop.h.nday1.SST.03010102-03510101.nc',
                         '/glade/work/wchapman/For_Tony//b.e21.B1850.f09_g17.CMIP6-esm-piControl.001.pop.h.nday1.SST.03510102-04010101.nc',
                          ]

for ee,fnfn in enumerate(sst_list): 
    print(ee)
    DSsst = xr.open_dataset(fnfn)
    if ee ==0: 
        DStot = DSsst
    else:
        DStot =xr.concat([DStot['SST'].to_dataset(),DSsst['SST'].to_dataset()],dim='time')
        
DStot.to_netcdf('/glade/work/wchapman/For_Tony//SST_camgrid_gathered.nc')
xr.merge([DStot,DSsst['REGION_MASK'],DSsst['UAREA'],DSsst['TAREA']]).to_netcdf('/glade/work/wchapman/For_Tony//SST_camgrid_gathered_Areas.nc')

0
1
2
3
4
5


In [11]:
DSolr =  xr.open_mfdataset(['/glade/scratch/wchapman/data_for_KJM/U_train_dask.nc',
 '/glade/scratch/wchapman/data_for_KJM/U_val_dask.nc',
 '/glade/scratch/wchapman/data_for_KJM/U_test_dask.nc'],parallel=True)
# DSolr = DSolr.chunk(chunks = {'time': 100, 'lat': 192, 'lon': 288})
DSolr.load()
DSolr.to_netcdf('/glade/scratch/wchapman/data_for_KJM/U_CESM2_100_400.nc')

In [4]:
sst_list = ['/glade/scratch/wchapman/data_for_KJM/U_train_dask.nc',
                        '/glade/scratch/wchapman/data_for_KJM/U_val_dask.nc',
                        '/glade/scratch/wchapman/data_for_KJM/U_test_dask.nc',
                          ]
for ee,fnfn in enumerate(sst_list): 
    print(ee)
    DSsst = xr.open_dataset(fnfn)
    if ee ==0: 
        DStot = DSsst
    else:
        DStot =xr.concat([DStot,DSsst],dim='time')
DStot.to_netcdf('/glade/scratch/wchapman/data_for_KJM/U_CESM2_100_400.nc')

0
1
2


## Get all the fields: 
- make sure they are all on the right temporal window

In [12]:
DS2 = xr.open_dataset('/glade/scratch/wchapman/data_for_KJM/MJO_CESM2_OLR_0100_0400.nc')
#change names below:
DS1 = xr.open_dataset('/glade/scratch/wchapman/data_for_KJM/U_anom_CESM2_100_400_repeats.nc')
DS1 = DS1.sel(time=slice(DS2['time'][0],DS2['time'][-1]))
DS1.to_netcdf('/glade/scratch/wchapman/data_for_KJM/U200_U850_CESM2_0100_0400.b.e21.B1850.f09_g17.CMIP6-esm-piControl.001.nc')

In [13]:
DS2 = xr.open_dataset('/glade/scratch/wchapman/data_for_KJM/MJO_CESM2_OLR_0100_0400.nc')
#change names below:
DS1 = xr.open_dataset('/glade/scratch/wchapman/data_for_KJM/FLUT_CESM2_100_400_repeats.nc')
DS1 = DS1.sel(time=slice(DS2['time'][0],DS2['time'][-1]))
DS1.to_netcdf('/glade/scratch/wchapman/data_for_KJM/FLUT_CESM2_0100_0400.b.e21.B1850.f09_g17.CMIP6-esm-piControl.001.nc')

In [14]:
DS2 = xr.open_dataset('/glade/scratch/wchapman/data_for_KJM/MJO_CESM2_OLR_0100_0400.nc')
#change names below:
DS1 = xr.open_dataset('/glade/scratch/wchapman/data_for_KJM/Z500_CESM2_100_400_repeats.nc')
DS1 = DS1.sel(time=slice(DS2['time'][0],DS2['time'][-1]))
DS1.to_netcdf('/glade/scratch/wchapman/data_for_KJM/Z500_CESM2_0100_0400.b.e21.B1850.f09_g17.CMIP6-esm-piControl.001.nc')

In [16]:
S2 = xr.open_dataset('/glade/scratch/wchapman/data_for_KJM/MJO_CESM2_OLR_0100_0400.nc')
#change names below:
DS1 = xr.open_dataset('/glade/scratch/wchapman/data_for_KJM/SST_CESM2_100_400_repeats.nc')
DS1 = DS1.sel(time=slice(DS2['time'][0],DS2['time'][-1]))
DS1.to_netcdf('/glade/scratch/wchapman/data_for_KJM/SST_CESM2_0100_0400.b.e21.B1850.f09_g17.CMIP6-esm-piControl.001.nc')

In [15]:
DS2.to_netcdf('/glade/scratch/wchapman/data_for_KJM/MJO_CESM2_0100_0400.b.e21.B1850.f09_g17.CMIP6-esm-piControl.001.nc')